In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mls_scf_tools.mls_pint import ureg
# plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]
# fmt: on

In [ ]:
import pandas as pd
from dataclasses import dataclass
import pint
import intervaltree
import math

In [ ]:
ureg = pint.UnitRegistry()
pint.set_application_registry(ureg)

In [ ]:
raw_data = pd.read_excel(
    "/users/livesey/corf/data/Oscar-Satellite-Frequencies-Earth-observation-MW-frequencies-2023-11-14.xlsx"
)

In [ ]:
raw_data

In [ ]:
raw_data.loc[raw_data["Id"] == 3956, "Bandwidth (MHz)"].astype(str)

In [ ]:
frequency_ranges = raw_data["Frequency (GHz)"].str.split("-", expand=True)
lower_limit = (
    frequency_ranges[0]
    .str.strip()
    .str.split(" ", expand=True)[0]
    .str.strip()
    .astype(float)
)
upper_limit = (
    frequency_ranges[1]
    .str.strip()
    .str.split(" ", expand=True)[0]
    .str.strip()
    .astype(float)
)
range_based_bandwidth = np.abs(upper_limit - lower_limit)

In [ ]:
data = raw_data.copy()
data["Frequency start"] = lower_limit
data["Frequency stop"] = upper_limit
data.loc[data["Frequency stop"].isnull(), "Frequency stop"] = data["Frequency start"]

In [ ]:
raw_data.loc[raw_data["Bandwidth (MHz)"] == "-"]

In [ ]:
bandwidth = data["Bandwidth (MHz)"].copy().astype(str).str.strip()
# .loc[data["Bandwidth (MHz)"] != "N/R", "Bandwidth (MHz)"].str.split(
#    " ", expand=True
# )[0]
bandwidth.loc[
    (bandwidth == "N/R") | (bandwidth == "-") | (bandwidth == "nan")
] = "0 MHz"
bandwidth_value = (
    bandwidth.str.split(" ", expand=True)[0].str.strip().astype(float) / 1e3
)
# Check that there is no confusion
questionable_row_flags = (bandwidth_value != 0) & (range_based_bandwidth > 1e-4)
#
display(
    raw_data[questionable_row_flags]
)  # , caption="Entries with redundant bandwidth inforamation")
# bandwidth_value[questionable_row_flags].to_list()
data.loc[~questionable_row_flags, "Frequency start"] -= (
    0.5 * bandwidth_value[~questionable_row_flags]
)
data.loc[~questionable_row_flags, "Frequency stop"] += (
    0.5 * bandwidth_value[~questionable_row_flags]
)

In [ ]:
subset = data.loc[(data["Frequency start"] > 200) & (data["Frequency stop"] < 220)]

In [ ]:
@dataclass
class OscarEntry:
    bounds: slice
    oscar_id: int
    satellite: str
    space_agency: str
    launch: str
    eol: str
    service: str
    sensing_mode: str
    nominal_frequency: str
    bandwidth: str
    polarization: str
    comment: str

    def __post_init__(self):
        """Just some tidying up after definition"""
        if self.bounds.stop == self.bounds.start:
            delta = (
                math.sqrt(math.ulp(self.bounds.stop.magnitude)) * self.bounds.stop.units
            )
            self.bounds = slice(self.bounds.start, self.bounds.stop + delta)


database = intervaltree.IntervalTree()
for row in data.to_dict("records"):
    entry = OscarEntry(
        bounds=slice(
            row["Frequency start"] * ureg.GHz, row["Frequency stop"] * ureg.GHz
        ),
        oscar_id=int(row["Id"]),
        satellite=row["Satellite"],
        space_agency=row["Space Agency"],
        launch=row["Launch "],
        eol=row["Eol"],
        service=row["Service"],
        sensing_mode=row["Sensing mode"],
        nominal_frequency=row["Frequency (GHz)"],
        bandwidth=row["Bandwidth (MHz)"],
        polarization=row["Polarisation"],
        comment=row["Comment"],
    )
    database[entry.bounds] = entry

In [ ]:
services = sorted(list(set([entry.data.service for entry in database])))
print(len(database))
print(services)

In [ ]:
for service in services:
    entries = [entry for entry in database if entry.data.service == service]
    if service == "MLS":
        print([entry.data.satellite for entry in entries])